# Inverted pendulu - custom neural network

Install dependencies

In [3]:
!pip3 install tensorflow
!pip3 install gym
!pip3 install keras
!pip3 install keras-rl2

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [4]:
import gym
import random

In [36]:
env = gym.make("CartPole-v0")
states = env.observation_space.shape[0] # get number of observation space variables
actions = env.action_space.n # get number of action space variables

In [37]:
actions

2

In [7]:
episodes = 10

for episode in range(1, episodes + 1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode {} Score {}'.format(episode, score))

env.close()

Episode 1 Score 36.0
Episode 2 Score 24.0
Episode 3 Score 9.0
Episode 4 Score 18.0
Episode 5 Score 34.0
Episode 6 Score 12.0
Episode 7 Score 10.0
Episode 8 Score 31.0
Episode 9 Score 21.0
Episode 10 Score 21.0


Create a Deeo Learning Model with Keras

In [38]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers.legacy import Adam

In [39]:
"""
states -> numbers of variables given as a output (input)
actions -> numebr of actions that neural network can do (output)
"""
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1, states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model


In [40]:
model = build_model(states, actions)

In [41]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 4)                 0         
                                                                 
 dense_9 (Dense)             (None, 24)                120       
                                                                 
 dense_10 (Dense)            (None, 24)                600       
                                                                 
 dense_11 (Dense)            (None, 2)                 50        
                                                                 
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


Build agent with Keras-RL

In [42]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy  # policy based rf
from rl.memory import SequentialMemory

In [43]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=5000, window_length = 1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)

    return dqn

In [44]:
model = build_model(states, actions)

In [54]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=20000, visualize=False, verbose=1)

2023-04-30 22:24:05.395106: W tensorflow/c/c_api.cc:300] Operation '{name:'dense_16_2/kernel/Assign' id:3996 op device:{requested: '', assigned: ''} def:{{{node dense_16_2/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_16_2/kernel, dense_16_2/kernel/Initializer/stateless_random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Training for 20000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 28:46 - reward: 1.0000

/home/kuba/.local/lib/python3.10/site-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2023-04-30 22:24:05.645123: W tensorflow/c/c_api.cc:300] Operation '{name:'dense_17/BiasAdd' id:3596 op device:{requested: '', assigned: ''} def:{{{node dense_17/BiasAdd}} = BiasAdd[T=DT_FLOAT, _has_manual_control_dependencies=true, data_format="NHWC"](dense_17/MatMul, dense_17/BiasAdd/ReadVariableOp)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-04-30 22:24:05.719386: W tensorflow/c/c_api.cc:300] Operation '{name:'total_25/Assign' id:4163 op device:{requested: '', assigned: ''} def:{{{node total_25/Assign}} = AssignVariableOp[_has_manual_control_

10000/10000 [==============================] - 55s 6ms/step - reward: 1.0000
56 episodes - episode_reward: 178.357 [41.000, 200.000] - loss: 5.271 - mae: 36.326 - mean_q: 72.871

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 55s 6ms/step - reward: 1.0000
done, took 110.645 seconds


In [55]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200
Episode 16: reward: 200.000, steps: 200
Episode 17: reward: 200.000, steps: 200
Episode 18: reward: 200.000, steps: 200
Episode 19: reward: 200.000, steps: 200
Episode 20: reward: 200.000, steps: 200
Episode 21: reward: 200.000, steps: 200
Episode 22: reward: 200.000, steps: 200
Episode 23: reward: 200.000, steps: 200
Episode 24: reward: 200.000, steps: 200
Episode 25: reward: 

In [28]:
_ = dqn.test(env, nb_episodes=5, visualize=True)
env.close()

Testing for 100 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200


KeyboardInterrupt: 

Reload agent from memory

In [56]:
dqn.save_weights('Training/dqn_weights.h5f', overwrite=True)

In [57]:
del model
del env
del dqn

In [58]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])

2023-04-30 22:26:25.256329: W tensorflow/c/c_api.cc:300] Operation '{name:'dense_19/bias/Assign' id:4680 op device:{requested: '', assigned: ''} def:{{{node dense_19/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_19/bias, dense_19/bias/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


In [59]:
dqn.load_weights('Training/dqn_weights.h5f')

2023-04-30 22:26:27.960270: W tensorflow/c/c_api.cc:300] Operation '{name:'count_31/Assign' id:4947 op device:{requested: '', assigned: ''} def:{{{node count_31/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](count_31, count_31/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


In [60]:
_ = dqn.test(env, nb_episodes=5, visualize=True)
env.close()

Testing for 5 episodes ...


2023-04-30 22:26:29.811316: W tensorflow/c/c_api.cc:300] Operation '{name:'dense_20/BiasAdd' id:4709 op device:{requested: '', assigned: ''} def:{{{node dense_20/BiasAdd}} = BiasAdd[T=DT_FLOAT, _has_manual_control_dependencies=true, data_format="NHWC"](dense_20/MatMul, dense_20/BiasAdd/ReadVariableOp)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200


In [53]:
env.close()